# Part 1

---


In [153]:
import numpy as np

In [154]:
# Reading the file
def prepare_data(file_path):
    """Prepare the raw data for training

    Args:
        file_path (string): input file path for preparation

    Returns:
        dictionary: state_to_idx - a mapping of each state to the index of the data.split('\n')
        dictionary: observation_to_idx - a mapping of each observation to the index of the data.split('\n')
        set: states - a set of unique states
        set: observations - a set of unique observations
        string: data - raw data

    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    # Collect unique states and observations
    states = set()
    observations = set()
    # split the data by new line - essentially split by comments/sentences
    for line in data.split('\n'):
        if line:
            state = line[line.rfind(' ') + 1:]
            observation = line[:line.rfind(' ')]
            # split a valid line by space - for observation and state
            states.add(state)
            observations.add(observation)

    observations.add("#UNK#")  # Include special token for unknown words

    # Create dictionary for states and observations - key is state or observation, value is index
    state_to_idx = {state: idx for idx, state in enumerate(states)}
    observation_to_idx = {obs: idx for idx, obs in enumerate(observations)}

    # the idx is important for the shape of the emission probabilities matrix (num_states x num_observations)

    return state_to_idx, observation_to_idx, states, observations, data


def estimate_b(train_data, state_to_idx, observation_to_idx, states, observations):
    """Estimate emission probabilities

    Args:
        train_data (string): raw data
        state_to_idx (dictionary): a mapping of each state to the index of the data.split('\n')
        observation_to_idx (dictionary): a mapping of each observation to the index of the data.split('\n')
        states (set): a set of unique states
        observations (set): a set of unique observations

    Returns:
        np.ndarray: emission_probabilities - a 2d array of emission probabilities (num_states x num_observations)
        note here that emission probabilities have the rows as the states and columns as the observations
    """
    k = 1
    # Initialize counters
    state_counts = np.zeros(len(states))  # count(y)
    # 2d array of emission counts (num_states x num_observations) for count(y -> x)
    emission_counts = np.zeros((len(states), len(observations)))

    # Count occurrences
    for line in train_data.split('\n'):  # get each line of the data
        if line:  # if the line is not empty
            state = line[line.rfind(' ') + 1:]
            observation = line[:line.rfind(' ')]
            # get the row index for this observation's state
            row_index = state_to_idx[state]
            # get the column index for this observation
            column_index = observation_to_idx[observation]

            # increase the number of occurrences for the state in the data
            state_counts[row_index] += 1
            # increase the number of occurrences for the observation with this state.
            emission_counts[row_index][column_index] += 1

    # Calculate emission probabilities
    emission_probabilities = (emission_counts) / (state_counts[:, None] + k)

    # Calculate for unknown words
    emission_probabilities[:, observation_to_idx["#UNK#"]
                           ] = k / (state_counts + k)

    return emission_probabilities


def predict(test_data, emission_probabilities, observation_to_idx, state_to_idx):
    """Perform sentiment analysis on a sequence of observations

    Args:
        test_data (string): a sequence of observations
        emission_probabilities (np.ndarray): emission probabilities
        observation_to_idx (dictionary): a mapping of each observation to the index of the data.split('\n')
        state_to_idx (dictionary): a mapping of each state to the index of the data.split('\n')

    Returns:
        list: predicted_tags - a list of predicted tags
    """
    predicted_tags = []
    observation_predicted_pairs = []
    for line in test_data.split('\n'):
        if line:  # if the line is not empty
            # remove the new line character
            observation = line.replace('\n', '')
            # get the index of the observation if it exists, otherwise get the index of the unknown token
            observation_idx = observation_to_idx.get(
                observation, observation_to_idx["#UNK#"])

            # get the index of the state with the highest probability
            max_prob_state_idx = np.argmax(
                emission_probabilities[:, observation_idx])
            # predicted_state = [state for state, idx in state_to_idx.items() if idx == max_prob_state_idx][0]

            for state, idx in state_to_idx.items():
                if idx == max_prob_state_idx:  # if the index of the state is the same as the index of the state with the highest probability
                    predicted_state = state
                    break

            predicted_tags.append(predicted_state)
            observation_predicted_pairs.append(
                observation + " " + predicted_state)
        else:
            observation_predicted_pairs.append("")

    return predicted_tags, observation_predicted_pairs


# def calculate_metrics(predicted_tags, gold_tags):
#     """Calculate precision, recall and F-score

#     Args:
#         predicted_tags (list): a nested list of predicted tags
#         gold_tags (list): list of gold standard tags (reference output)

#     Returns:
#         float: Precision
#         float: Recall
#         float: F-score
#     """
#     correct_entities = 0
#     predicted_entities = 0
#     gold_entities = 0

#     predicted_entity_start = False
#     predicted_gold_start = False

#     for i in range(len(predicted_tags)):
#         predicted_tag = predicted_tags[i]
#         gold_tag = gold_tags[i]

#         # current tags
#         if gold_tag != 'O':
#             # add correct entities
#             if gold_tag == predicted_tag:
#                 correct_entities += 1

#             # check if an entity has started
#             if gold_tag == "B-positive" or gold_tag == "B-negative" or gold_tag == "B-neutral":
#                 gold_entities += 1
#                 predicted_gold_start = True

#             # check if an entity has started without a B tag
#             elif (gold_tag == "I-positive" or gold_tag == "I-negative" or gold_tag == "I-neutral") and predicted_gold_start == False:
#                 gold_entities += 1
#                 predicted_gold_start = True
#         else:
#             predicted_gold_start = False  # reset the flag if the current tag is 'O'

#         if predicted_tag != 'O':
#             # check if an entity has started
#             if predicted_tag == "B-positive" or predicted_tag == "B-negative" or predicted_tag == "B-neutral":
#                 predicted_entities += 1
#                 predicted_entity_start = True

#              # check if an entity has started without a B tag
#             elif (predicted_tag == "I-positive" or predicted_tag == "I-negative" or predicted_tag == "I-neutral") and predicted_entity_start == False:
#                 predicted_entities += 1
#                 predicted_entity_start = True
#         else:
#             predicted_entity_start = False

#     Precision = correct_entities / predicted_entities
#     Recall = correct_entities / gold_entities
#     F = 2 / (1/Precision + 1/Recall)

#     return Precision, Recall, F, correct_entities, predicted_entities, gold_entities


def gold_labels(data):
    """Get the gold labels from the data

    Args:
        data (string): raw data

    Returns:
        list: gold_labels - a list of gold labels
    """
    gold_labels = []
    for line in data.split('\n'):
        if line:
            observation, state = line.split(' ')
            gold_labels.append(state)
    return gold_labels

In [155]:
# for ES dataset

# prepare data
es_state_to_idx, es_observation_to_idx, es_states, es_observations, es_train_data = prepare_data(
    ".\\ES\\train")

# probability of each state
es_emission_probabilities = estimate_b(
    es_train_data, es_state_to_idx, es_observation_to_idx, es_states, es_observations)


# predict
with open(".\\ES\\dev.in", "r") as f:
    es_test_data = f.read()  # read in the file as a string

# predict the states for the test data
es_predicted_states, es_observation_predicted_tags = predict(
    es_test_data, es_emission_probabilities, es_observation_to_idx, es_state_to_idx)


# Write to dev.p1.out
with open(".\\ES\\dev.p1.out", "w") as f:
    for i in range(len(es_observation_predicted_tags)):
        f.write(es_observation_predicted_tags[i] + "\n")

In [156]:
# for RU dataset

# prepare data
ru_state_to_idx, ru_observation_to_idx, ru_states, ru_observations, ru_train_data = prepare_data(
    ".\\RU\\train")


# probability of each state
ru_emission_probabilities = estimate_b(
    ru_train_data, ru_state_to_idx, ru_observation_to_idx, ru_states, ru_observations)


# predict
with open(".\\RU\\dev.in", "r") as f:
    ru_test_data = f.read()  # read in the file as a string

# predict the states for the test data
ru_predicted_states, ru_observation_predicted_tags = predict(
    ru_test_data, ru_emission_probabilities, ru_observation_to_idx, ru_state_to_idx)


# Write to dev.p1.out
with open(".\\RU\\dev.p1.out", "w") as f:
    for i in range(len(ru_observation_predicted_tags)):
        f.write(ru_observation_predicted_tags[i] + "\n")

# Evaluation for ES

#### Entity in gold data: 229

#### Entity in prediction: 1466

#### Correct Entity : 178

-   Entity precision: 0.1214
-   Entity recall: 0.7773
-   Entity F: 0.2100

#### Correct Sentiment : 97

-   Sentiment precision: 0.0662
-   Sentiment recall: 0.4236
-   Sentiment F: 0.1145

---

# Evaluation for RU

#### Entity in gold data: 389

#### Entity in prediction: 1816

#### Correct Entity : 266

-   Entity precision: 0.1465
-   Entity recall: 0.6838
-   Entity F: 0.2413

#### Correct Sentiment : 129

-   Sentiment precision: 0.0710
-   Sentiment recall: 0.3316
-   Sentiment F: 0.1170


---
# Part 2
---


In [157]:
# Helper Functions

def prepare_transition_data(file_path):
    with open(file_path, "r") as f:
        data = f.read()
        
    states = set() # a unique set of states
    states.add("START")
    
    for line in data.split("\n"):
        if line:
            state = line[line.rfind(" ") + 1:]
            states.add(state) 

    states.add("STOP")
    state_to_idx = {state: idx for idx, state in enumerate(states)} # a dictionary mapping states to indices
    return states, state_to_idx, data



def estimate_a(train_data, state_to_idx, states):
    number_of_states = len(states)
    transitions = np.zeros((number_of_states, number_of_states)) # a matrix that counts the number of transitions: count(u; v)
    total_state_counts = np.zeros(number_of_states) # counts the total number of states in the data: count(u)      
    
    # count occurences
    start = True
    for line in train_data.split("\n"):
        if line and start: # beginning of a section
            u = "START"
            v = line[line.rfind(" ") + 1:]
            start = False
            prev = v
            
        elif line and not start: # middle of a section
            u = prev
            v = line[line.rfind(" ") + 1:]
            prev = v
        
        else: # end of a section - indicated by empty line
            u = prev
            v = "STOP"
            start = True
            
        # update the counts
        # print(u,v)
        transitions[state_to_idx[u], state_to_idx[v]] += 1
        total_state_counts[state_to_idx[u]] += 1
    
    # estimate transition probabilities
    a = np.zeros((number_of_states, number_of_states))
    for u in states:
        for v in states:
            # q(yi|yi−1) = Count(yi−1, yi) / Count(yi−1)
            
            # account for division by 0
            if total_state_counts[state_to_idx[u]] == 0:
                a[state_to_idx[u], state_to_idx[v]] = 0
            else:
                a[state_to_idx[u], state_to_idx[v]] = transitions[state_to_idx[u], state_to_idx[v]] / total_state_counts[state_to_idx[u]]
    return a


def r(tags, state_to_idx, observation_to_idx, emission_probabilities, transition_probabilities):
    """calculate joint probability of a sequence of tags

    Args:
        tags (list): a sequence of tags
        state_to_idx (numpy.ndarray): a dictionary mapping states to indices
        observation_to_idx (numpy.ndarray): a dictionary mapping observations to indices
        emission_probabilities (numpy.ndarray): 2D matrix of emission probabilities
        transition_probabilities (numpy.ndarray): 2D matrix transition probabilities
    """
    for i in range(len(tags)):
        y = tags[i]
        if i == 0:
            p = transition_probabilities[state_to_idx["START"], state_to_idx[y]] * emission_probabilities[state_to_idx[y], observation_to_idx[y]]
        else:
            y_prev = tags[i-1]
            p *= transition_probabilities[state_to_idx[y_prev], state_to_idx[y]] * emission_probabilities[state_to_idx[y], observation_to_idx[y]]
            
    return p
    
def viterbi(transition_prob, emission_prob, state_to_idx, state_to_idx_observations, observation_to_idx, observations):
    """Use Viterbi algorithm to find the most likely sequence of states

    Args:
        transition_prob (numpy.ndarray): 2D matrix of transition probabilities
        emission_prob (numpy.ndarray): 2D matrix of emission probabilities
        state_to_idx (dictionary): a dictionary mapping states (does not include start and stop) to indices
        observation_to_idx (dictionary): a dictionary mapping observations to indices
        observations (list): a sequence of observations from the train data
        
    Returns:
        best_path_states (list): the most likely sequence of states for the observations
    """
    number_of_observations = len(observations)
    
    # initialize tabulation table for bottom up dynamic programming, + 2 to account for start and stop
    dp = np.zeros((number_of_observations+2, len(state_to_idx))) # (number of observations, number of states) - dp[i, j] = most likely sequence of states up to observation i with state j
    
    start_idx = state_to_idx["START"]
    stop_idx = state_to_idx["STOP"]
    
    
    # (step 1)
    # Base case - k = 0 is the START where the first observation is k = 1 -- each row in the dp table is an observation
    dp[0, start_idx] = 1 # start state
    
    
    # (step 2)
    for j in range(1, number_of_observations-2): # number_of_observations - 2 because we don't want to include the last row which is STOP
        for u in state_to_idx:
            u_idx = state_to_idx[u] # index of state u which is the column index, u is the previous state
            for v in state_to_idx:
                if v == "START" or v == "STOP": # skip start and stop states
                    continue
                observation_idx = observation_to_idx[observations[j]] # index of observation
                v_idx = state_to_idx[v] # index of state v which is the column index, v is the current state
                v_idx_observations = state_to_idx_observations[v] # index of state v for the observation_to_idx matrix
                # recurrence relation
                dp[j, v_idx] = max(dp[j, v_idx], dp[j-1, u_idx] * emission_prob[v_idx_observations, observation_idx] * transition_prob[u_idx, v_idx] )
    
    # (step 3)        
    # account for stop state
    for i in range(len(dp[number_of_observations-2])): # iterate through the second to last row
        if i == stop_idx or i == start_idx: # skip start and stop states
            continue
        dp[number_of_observations-1, stop_idx] = max(dp[number_of_observations-1, stop_idx], dp[number_of_observations-2, i] * transition_prob[i, stop_idx])
    
    # output predicted sequence of states
    print(dp)
    best_path_states = []
    for index in range(dp.shape[0]):
        best_path_states.append(np.argmax(dp[index]))
    
    return best_path_states
        

    
    
            

In [158]:
# for ES

es_t_states, es_t_state_to_idx, es_train_data = prepare_transition_data(".\\ES\\train")


es_transition_probabilities = estimate_a(es_train_data, es_t_state_to_idx, es_t_states) # the state to idx should not include start and stop to ensure correct indices

# write to dev.p2.out

es_train_data = es_train_data.split("\n")

observations = []

with open(".\\ES\\dev.p2.out", "w") as f:
    # grab each chunk and use viterbi to predict the sequence of tags
    for line in es_train_data:
        if not line:
            # empty line indicates the end of a sentence
            predicted_states = viterbi(es_transition_probabilities, es_emission_probabilities, es_t_state_to_idx, es_state_to_idx, es_observation_to_idx, observations)
            print(predicted_states)
            # write to file
            for i in range(len(predicted_states)):
                f.write(observations[i] + " " + predicted_states[i] + "\n")
            f.write("\n")
            
            observations = [] # reset observations
        
        else:
            word, tag = line.split(" ")
            observations.append(word)






    

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00]
 [0.00000000e+00 8.31790117e-04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.39542432e-06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 3.06456350e-09 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.21518337e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 3.79552291e-14 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 4.16777207e-17 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00

TypeError: can only concatenate str (not "numpy.int64") to str